<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice Spark Lab

---



### Some useful Spark resources

- [Spark CSV](https://github.com/databricks/spark-csv)
- [Pyspark programming guide](https://spark.apache.org/docs/0.9.0/python-programming-guide.html)
- [Download and run Spark](https://github.com/mahmoudparsian/pyspark-tutorial/blob/master/howto/download_install_run_spark.md)

---

**In this lab, we will use Spark to dig into the Bay Area Bike Share data.**

Our goal is to calculate the average number of trips per hour, using the Caltrain Station as starting point.

In [1]:
import pyspark as ps    # for the pyspark suite
import warnings         # for displaying warning
from pyspark.sql import SQLContext

In [2]:
try:
    # we try to create a SparkContext to work locally on all cpus available
    sc = ps.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    # give a warning if SparkContext already exists (for use inside pyspark)
    warnings.warn("SparkContext already exists in this scope")

Just created a SparkContext


### 1. Load the Bay Area Bike Share trip data


> **Note:** This dataset stems from: http://www.bayareabikeshare.com/open-data

In [3]:
# A:
trips = sc.textFile('./data/201508_trip_data.csv')

In [4]:
trips.first()

'Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code'

### 2. What kind of object is the data loaded as?

In [5]:
type(trips)

pyspark.rdd.RDD

### 3. Split csv lines

In spark, we can build complex pipelines that only get executed when we ask to collect them.

In a python pipeline the calculation is immediately executed, but with spark the pipeline definition and execution are separate steps.

In other words, we can define the pipeline with all its steps, and only when we call `collect` will the data flow through it. In order to get familiar with this new workflow, we will start with small steps to build our pipeline.

**Apply a map to trips that splits each line at commas and save that to a an RDD.**

> **Hint:** if you want to check that you're doing things right, you can collect the result and display the first few lines.

In [6]:
trips.take(5)

['Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code',
 '913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139',
 '913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032',
 '913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107',
 '913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113']

In [7]:
trips = trips.map(lambda x: x.split(',') )
#  don't forget to collect!

In [8]:
trips.take(2)

[['Trip ID',
  'Duration',
  'Start Date',
  'Start Station',
  'Start Terminal',
  'End Date',
  'End Station',
  'End Terminal',
  'Bike #',
  'Subscriber Type',
  'Zip Code'],
 ['913460',
  '765',
  '8/31/2015 23:26',
  'Harry Bridges Plaza (Ferry Building)',
  '50',
  '8/31/2015 23:39',
  'San Francisco Caltrain (Townsend at 4th)',
  '70',
  '288',
  'Subscriber',
  '2139']]

### 4. Filter for Caltrain station

In Spark we can also create filters using the `filter` method.

**Select station number 70 by filtering on the 5th column.** 

We will do all the following analysis just on this station, which corresponds to the most popular starting point. Save this to a variable called `station_70`.

In [9]:
headers = trips.first()

In [10]:
station_70 = trips.filter(lambda x: x[4]=='70')

In [11]:
station_70.take(2)

[['913429',
  '902',
  '8/31/2015 21:07',
  'San Francisco Caltrain (Townsend at 4th)',
  '70',
  '8/31/2015 21:22',
  'Broadway St at Battery St',
  '82',
  '501',
  'Subscriber',
  '94133'],
 ['913426',
  '481',
  '8/31/2015 21:06',
  'San Francisco Caltrain (Townsend at 4th)',
  '70',
  '8/31/2015 21:14',
  'Market at 4th',
  '76',
  '542',
  'Subscriber',
  '95054']]

In [12]:
# A:
# station_70 = trips.filter(...

### 5. Trips by day - hour (mapper)

Let's analyse the trips by the hour. We can do this by performing a map reduce job in Spark. First we will need to emit tuples with a count of 1 for each (date, hour) key, and then we will sum the counts by key.

**Emit tuple of ((date, hour), 1), applying a map to `station_70` that extracts the relevant data from each line.**

In [13]:
trips.take(2)

[['Trip ID',
  'Duration',
  'Start Date',
  'Start Station',
  'Start Terminal',
  'End Date',
  'End Station',
  'End Terminal',
  'Bike #',
  'Subscriber Type',
  'Zip Code'],
 ['913460',
  '765',
  '8/31/2015 23:26',
  'Harry Bridges Plaza (Ferry Building)',
  '50',
  '8/31/2015 23:39',
  'San Francisco Caltrain (Townsend at 4th)',
  '70',
  '288',
  'Subscriber',
  '2139']]

In [14]:
# A:
# trips_by_day_hour = station_70.map(...

In [16]:
station_70.map(lambda x: x[2]).take(2)

['8/31/2015 21:07', '8/31/2015 21:06']

In [17]:
station_70.map(lambda x: x[2].split(' ')).take(2)

[['8/31/2015', '21:07'], ['8/31/2015', '21:06']]

In [18]:
station_70.map(lambda x: (x[2].split(' ')[0],x[2].split(' ')[1])).take(2)

[('8/31/2015', '21:07'), ('8/31/2015', '21:06')]

In [19]:
station_70.map(lambda x: (x[2].split(' ')[0],x[2].split(' ')[1].split(':')[0])).take(2)

[('8/31/2015', '21'), ('8/31/2015', '21')]

In [20]:
station_70.map(lambda x: ((x[2].split()[0],x[2]\
.split()[1].split(':')[0]),1)).take(10)

[(('8/31/2015', '21'), 1),
 (('8/31/2015', '21'), 1),
 (('8/31/2015', '20'), 1),
 (('8/31/2015', '19'), 1),
 (('8/31/2015', '18'), 1),
 (('8/31/2015', '18'), 1),
 (('8/31/2015', '18'), 1),
 (('8/31/2015', '18'), 1),
 (('8/31/2015', '18'), 1),
 (('8/31/2015', '18'), 1)]

In [21]:
trips_by_day_hour = station_70 \
    .map(lambda x: ((x[2].split()[0],x[2] \
    .split()[1].split(':')[0]),1)) \
    .reduceByKey(lambda x, y: x+y) \
    

In [22]:
trips_by_day_hour.take(10)

[(('8/31/2015', '21'), 2),
 (('8/31/2015', '20'), 1),
 (('8/31/2015', '19'), 1),
 (('8/31/2015', '17'), 7),
 (('8/31/2015', '16'), 3),
 (('8/31/2015', '14'), 2),
 (('8/31/2015', '10'), 4),
 (('8/31/2015', '9'), 6),
 (('8/31/2015', '8'), 25),
 (('8/30/2015', '20'), 1)]

### 6. Trips by day - hour (reducer)

Use the `reduceByKey` method to obtain the number of trips per (day, hour).

In [23]:
trips_by_day_hour = trips_by_day_hour.\
reduceByKey(lambda x,y:x+y)

In [33]:
trips_by_day_hour.take(10)

[(('8/31/2015', '21'), 2),
 (('8/31/2015', '20'), 1),
 (('8/31/2015', '19'), 1),
 (('8/31/2015', '17'), 7),
 (('8/31/2015', '16'), 3),
 (('8/31/2015', '14'), 2),
 (('8/31/2015', '10'), 4),
 (('8/31/2015', '9'), 6),
 (('8/31/2015', '8'), 25),
 (('8/30/2015', '20'), 1)]

### 7. Trips by hour (mapper)

Let's further group the trips by hour. We'll do this with a second Map Reduce job.

First we will discard the day and emit tuples of (hour, count). You can achieve this with a map.

In [25]:
trips_by_day_hour.take(2)

[(('8/31/2015', '21'), 2), (('8/31/2015', '20'), 1)]

In [29]:
trips_by_day_hour_7 = trips_by_day_hour.map(lambda x: (x[0][1],x[1]))\
.reduceByKey(lambda x,y:x+y)

In [31]:
trips_by_day_hour_7.take(5)

[('21', 311), ('20', 498), ('19', 1366), ('17', 2023), ('16', 807)]

### 8. Trips by hour (reducer)

Now calculate the average number of trips by hour using the `combineByKey` method.

> You can find a suggestion on how to do it [here](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/).

In [34]:
trips_by_day_hour.map(lambda x: (x[0][1],1)).take(2)

[('21', 1), ('20', 1)]

In [35]:
trips_by_day_hour.map(lambda x: (x[0][1],1)).\
reduceByKey(lambda x,y: x+y).take(10)

[('21', 190),
 ('20', 231),
 ('19', 279),
 ('17', 307),
 ('16', 298),
 ('14', 184),
 ('10', 306),
 ('9', 319),
 ('8', 286),
 ('12', 213)]

### 9. `collect()` the results.
We can finally collect our result and sort them.

In [ ]:
# A:

### 10. [Bonus] Using the Spark `sqlContext`

Besides the SparkContext, Spark also exposes a sqlContext that allows us to perform SQL queries on an RDD object.

A SQLContext is also already created for you. Do not create another or unspecified behavior may occur. As you can see below, the sqlContext provided is a HiveContext.

**Run a query using the sqlContext to obtain the average duration of a trip originating from the Caltrain station.**

*Note: you might have to rename the columns*

In [ ]:
# A:
tripsSql = sqlContext.read.format('com.databricks.spark.csv').options(header='true',
                inferschema='true').load('./data/201508_trip_data.csv')